In [14]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, ShuffleSplit
import rdkit.Chem as Chem
import descriptors
import Ipynb_importer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
import math
warnings.filterwarnings("ignore")

In [16]:
data = pd.read_excel('778DATA.xlsx') 
data['Mols'] = data['SMILES'].apply(Chem.MolFromSmiles)
data['Mols'] = data['Mols'].apply(Chem.AddHs)
bond_types,x_SOB = descriptors.literal_bag_of_bonds(list(data['Mols']))
x_Estate = descriptors.truncated_Estate_fingerprints(list(data['Mols']))
x_cds = descriptors.custom_descriptor_set(list(data['Mols']))
SEC = np.concatenate((x_cds, x_SOB, x_Estate), axis=1)
CBD = pd.read_csv('CBD_descriptors/CBD.csv',header = None) 
X = np.concatenate((CBD, SEC), axis=1)
print(X.shape)

Y = np.array(list(data['BDE']))
print(Y.shape)

['N=O', 'N-O', 'C-N', 'C:C', 'C:N', 'N:N', 'C-H', 'H-N', 'N=N', 'C-O', 'N#N', 'C-C', 'N-N', 'C=O', 'C#C', 'C#N', 'H-O', 'N:O', 'C:O', 'C=N', 'C=C', 'O-O']
(778, 148)
(778,)


In [22]:
def normalize(X, a, b):
    return (X - a)/(b - a)

def denormalize(X, a, b):
    return X * (b - a) + a

def PADRE(X1,X2):
    if list(X1) != list(X2):
        X1X2 = X1-X2
        return np.concatenate([X1, X2, X1X2], axis=0)
    else:
        return 0
def PADRE_test(X1,X2):
    if list(X1) != list(X2):
        X1X2 = X1-X2
        return np.concatenate([X1, X2, X1X2], axis=0)
    else:
        return np.zeros((444))
    
pre_error = []
std_list = []
r2_list = []
MAE_list = []
RMSE_list = []
MAE_list_train = []
MAE_list_test = []
RMSE_list_train =[]
RMSE_list_test =[]
r2_list_train = []
r2_list_test = []
MAError_train_aver_list = []
MAError_test_aver_list = []
RMSEerror_train_aver_list =[]
RMSEerror_test_aver_list =[]
r2_train_aver_list =[]
r2_test_aver_list =[]

RS = ShuffleSplit(n_splits=20, test_size=0.2, random_state=5)
A = 1
for train_ind, test_ind in RS.split(X):
    X_train = X[train_ind]
    Y_train = Y[train_ind]
    X_test = X[test_ind]
    Y_test = Y[test_ind]
    
    padre_ = [PADRE(x,y) for x in X_train for y in X_train]
    bde_ = [x-y for x in Y_train for y in Y_train]

    h = 0
    padre = []
    bde_y = []
    for j in padre_:
        if j is not 0 :
            padre.append(j)
            bde_y.append(bde_[h])
        h = h + 1

    padre = np.array(padre)
    bde_y = np.array(bde_y)


    p_max = np.amax(padre); p_min = np.amin(padre)
    b_max = np.amax(bde_y); b_min = np.amin(bde_y)
    padre = normalize(padre,p_min,p_max)
    bde_y = normalize(bde_y,b_min,b_max)
    bde_mean = np.mean(Y_train)

    x_train, x_test, y_train, y_test = train_test_split(padre, bde_y, test_size=0.1, random_state=2)

    y_train_d = denormalize(y_train,b_min,b_max)
    y_test_d = denormalize(y_test,b_min,b_max)
   
    model = xgb.XGBRegressor(n_estimators=10, max_depth=9,learning_rate= 0.3,min_child_weight=5)
    model.fit(x_train,y_train)

    y_train_pred = denormalize(model.predict(x_train),b_min,b_max)
    y_test_pred = denormalize(model.predict(x_test),b_min,b_max)

    MAError_train = mean_absolute_error(y_train_d,y_train_pred)
    MAE_list_train.append(MAError_train)
    MAError_test = mean_absolute_error(y_test_d,y_test_pred)
    MAE_list_test.append(MAError_test)

    RMSEerror_train = math.sqrt(mean_squared_error(y_train_d,y_train_pred))
    RMSE_list_train.append(RMSEerror_train)
    RMSEerror_test = math.sqrt(mean_squared_error(y_test_d,y_test_pred))
    RMSE_list_test.append(RMSEerror_test)

    r2_train = r2_score(y_train_d,y_train_pred)
    r2_list_train.append(r2_train)
    r2_test = r2_score(y_test_d,y_test_pred)
    r2_list_test.append(r2_test)
    print('r2_val:',r2_test)
    if A == 21:
        print('rmse_val:',RMSE_list_test)
        print('r2_val_mean:',np.mean(r2_list_test))

    pre_mean = []
    predictions = []
    list1 = []
    list2 = []
    
    model = xgb.XGBRegressor(n_estimators=10, max_depth=20,learning_rate= 0.3,min_child_weight=10)
    model.fit(padre, bde_y)
    
    for k in range(len(X_test)):  

        padre_test = normalize(np.array([PADRE_test(X_test[k],x) for x in X_train]),p_min,p_max)      
        predictions = denormalize(model.predict(padre_test),b_min,b_max) + Y_train
        mean1 = np.mean(predictions)
        std1 = np.std(predictions,axis=0)
        pre_mean.append(mean1)
        pre_error.append(std1)
    std_list.append(std1)
    MAE = mean_absolute_error(Y_test, pre_mean)
    MAE_list.append(MAE)
    RMSE = math.sqrt(mean_squared_error(Y_test, pre_mean))
    RMSE_list.append(RMSE)
    r2 = r2_score(Y_test, pre_mean )
    r2_list.append(r2)
    print('r2_test:',r2)
    print('MAE_test:',MAE)
    print('RMSE_test:',RMSE)
    print('\n')
    A = A + 1

r2_val: 0.9853194709726878
r2_test: 0.9729420194071926
MAE_test: 7.859124097481783
RMSE_test: 12.800882067406205


r2_val: 0.9855006203288872
r2_test: 0.9680848181087633
MAE_test: 8.529543604591423
RMSE_test: 12.23852893408841


r2_val: 0.9859980594994013
r2_test: 0.9672721230713854
MAE_test: 9.098527579792949
RMSE_test: 14.383647446285083


r2_val: 0.9858876441318519
r2_test: 0.9714611522168572
MAE_test: 7.6151041929259105
RMSE_test: 12.176157550640426


r2_val: 0.9852638065817199
r2_test: 0.9794983405962964
MAE_test: 7.727037438363377
RMSE_test: 11.412163336209458


r2_val: 0.985270352822999
r2_test: 0.9707447075041427
MAE_test: 8.902897919587236
RMSE_test: 13.869278681515793


r2_val: 0.9859119312596485
r2_test: 0.9725614362077735
MAE_test: 8.730060807043802
RMSE_test: 12.66417269566686


r2_val: 0.9849942759419094
r2_test: 0.9733473758006012
MAE_test: 8.611689218857997
RMSE_test: 13.831828042398996


r2_val: 0.9851957332463175
r2_test: 0.9473728674950894
MAE_test: 9.023156091618214